# About:

In [1]:
%reload_ext autoreload
%autoreload 2

import os, sys

sys.path.append(os.path.abspath(os.pardir))

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
COUNTRY_NAME = os.environ.get("COUNTRY_NAME")
SPOKEN_LANGUAGE = "French"

table_column_name = "TexteCourt"

In [4]:
from general_config import COUNTRY_NAMES_LIST

# validate COUNTRY_NAME
if COUNTRY_NAME in COUNTRY_NAMES_LIST:
    print(COUNTRY_NAME, 'country name OK')

MLI Mali country name OK


In [5]:
from llm_setup import *

# Get all documents for the selected country

In [6]:
(df, documents) = get_country_legal_docs(COUNTRY_NAME)

/home/andrei/Ferdi_LLM/postgres_connection.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


# Get all documents for the selected country

In [8]:
embedding_model = get_voyage_ai_embedding_model()

vector_index = get_vector_db_from_documents(documents, COUNTRY_NAME, embedding_model, chunk_size=1024)

loading from local


In [9]:
doc_titles = pd.Series(list(set([v.get('title') for k,v in vector_index.vector_store.to_dict()['metadata_dict'].items()])))
doc_titles[0]

'Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (SGG)'

# Set the llm
the llm will be used to augment the retrieved data from the vector_index

In [16]:
llm = get_open_ai_llm()

# Get Relationships between texts

In [17]:
from difflib import get_close_matches

def find_most_similar_string(target, string_list):
    # Using difflib's get_close_matches function to find the closest match
    # It returns a list of close matches; we take the first (most similar) one
    closest_matches = get_close_matches(target, string_list, n=1, cutoff=0.0)
    return closest_matches[0] if closest_matches else None

def get_text_relationship(vector_index, doc_name, llm, config):
    vector_query_engine = create_vector_engine(vector_index=vector_index, doc_name=doc_name, llm=llm, config=config, operator="==")
    
    # What other documents from the knowledge base does it relate to or modify?
    # To answer this, take into account all the contents of the documents, and the metadata.
    # If the title is not part of the knowledge database, please respond with None.
    # Keep the title only in the final response, and no other data or explanation.

    response = vector_query_engine.query(
        f"""
        You are a helpful assistant who can read and analyze legal documents.
        Consider this text: '{doc_name}' and the contents and the metadata of the document with this title '{doc_name}'.
        Give me a list titles of legal documents it relates to.
        For each of the found documents, specify (separaed by comma):
         - the type of law
         - the year of the document
         - what's the relationship with the document: modifies, amends, abrogates, extends
        Separate them by new line.
        If you cannot decide, return None.
        """
    )

    print(doc_name, len(response.source_nodes))
    print([node.score for node in response.source_nodes])
    print([node.text for node in response.source_nodes])

    return response.response

In [18]:
response = get_text_relationship(
    vector_index=vector_index,
    doc_name = doc_titles[2],
    llm=llm,
    config={
        "similarity_top_k": 20,
        "rerank_top_k": 2

    }
    )

print(response)

img_Loi n°2004-037 (02.08.2004) Code pétrolier 2004 (Droit-Afrique) 2
[0.89453125, 0.89453125]
['^ CFA and imprisonment of six (6) months to two (2) years, or one of these two penalties is imposed on Petroleum Agreement holders who do not inform the National Directorate of all accidents and cases of identified dangers during the Petroleum Operations in accordance with the provisions of the present Law.\n\n### Article 90\nInfractions against the provisions of the present Law and its implementing regulations must be noted by official reports drawn up by agents of the National Directorate of the competent administrative authority.\n\n### Article 91\nShall be punished by imprisonment from 1 month to 1 year and a fine of 100,000 to 1,000,000 Francs CFA or one of these penalties only any person who, without being a holder of an Authorization to Research or Authorization to Develop or being mandated by a holder, exercises the rights conferred by such Authorisations.\n\n### Article 92\nShall b

In [19]:
texte_code_arborescence = list()

retries = 5

for doc in doc_titles:
    trial = 0

    while trial < retries:
        try:
            categ = get_text_relationship(
                vector_index=vector_index,
                doc_name = doc,
                llm=llm,
                config={
                    "similarity_top_k": 20,
                    "rerank_top_k": 5

                }
                )
            
            error = False

            break
        except Exception as e:
            print(e)

            error = True
            trial += 1

    if error:
        categ = "non_determined"

    texte_code_arborescence.append((doc, categ))    

Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (SGG) 2
[0.90625, 0.859375]
["DECRET N°00-050/PM-RM DU 10 FEVRIER 2000 PORTANT APPROBATION DES\nMODIFICATIONS A LA CONVENTION D’ETABLISSEMENT-TYPE POUR LA PROSPECTION, LA\nRECHERCHE ET L’EXPLOITATION DES SUBSTANCES MINERALES.\n\nLE PREMIER MINISTRE,\n\nVu la Constitution ;\n\nVu l’Ordonnance N°99-032/P-RM du 19 août 1999 portant Code Minier en République du Mali modifiée par\nl’Ordonnance N°00-013/P-RM du 10 février 2000 ;\n\nVu le Décret N°99-255/P-RM du 15 septembre 1999 fixant les modalités d’application de l’Ordonnance N°99-\n032/P-RM du 19 Août 1999 portant Code Minier en République du Mali ;\n\nVu le Décret N°97-263/P-RM du 13 septembre 1997 portant nomination d’un Premier ministre ;\n\nVu le Décret N°97-282/P-RM du 16 septembre 1997 portant nomination des membres du Gouvernement ;\n\nDECRETE :\n\nARTICLE 1er : Sont approuvées les modifications des articles 11 ; 18.2, k) ; 18.7, f) et 18.8, f) de la Conventi

In [21]:
import re
from llama_index.core.llms import ChatMessage

for doc in doc_titles:

    split_text = re.split(r'\(\d{2}\.\d{2}\.\d{4}\)', doc)

    if len(split_text) > 1:
        messages = [
            ChatMessage(
                role="assistant", content= \
                    f"""Considering only this title {split_text[1]}, what code does it modify.
                    Keep the title of the code, the domain of law, the year of the document, separated by comma.
                    And give no further information or explanation. Add ", modifie" at the end.
                    """
            ),
        ]
        resp = llm.chat(messages)

        texte_code_arborescence.append((doc, resp.message.content))    


In [29]:
# texte_code_arborescence[doc_titles.to_list().index("Décret n°1996-179 (19.06.1996) Application Office malien de l_habitat 1996 (SGG)")]
list_of_texte_code_arborescence = list()
for text in texte_code_arborescence:
    for rel in text[1].split("\n"):
        cols = rel.split(", ")

        if len(cols) != 4:
            list_of_texte_code_arborescence.append([text[0]] + 4 * [None])
        else:
            list_of_texte_code_arborescence.append([text[0]] + cols)

In [30]:
list_of_texte_code_arborescence

[['Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (SGG)',
  'Ordonnance N°99-032/P-RM',
  'droit minier',
  '1999',
  'modifies'],
 ['Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (SGG)',
  'Ordonnance N°00-013/P-RM',
  'droit minier',
  '2000',
  'amends'],
 ['Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (SGG)',
  'Décret N°99-255/P-RM',
  'droit minier',
  '1999',
  'fixes modalities of application'],
 ['Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (SGG)',
  'Décret N°97-263/P-RM',
  'droit minier',
  '1997',
  'nominates Prime Minister'],
 ['Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (SGG)',
  'Décret N°97-282/P-RM',
  'droit minier',
  '1997',
  'nominates Government members'],
 ['Décret n°1999-255 (15.09.1999) Application Code minier 1999 (SGG)',
  None,
  None,
  None,
  None],
 ['img_Loi n°2004-037 (02.08.2004) Code pétrolier 2004 (Droit-Afri

In [45]:
df_TexteCodeArborescence = pd.DataFrame(list_of_texte_code_arborescence)
df_TexteCodeArborescence = df_TexteCodeArborescence[~df_TexteCodeArborescence[1].isna()]

df_TexteCodeArborescence.to_csv(f"../output/TexteCodeArborescence_{COUNTRY_NAME}.csv", index=True)

df_TexteCodeArborescence

,0,1,2,3,4
0,Décret n°2000-050 (10.02.2000) Modification Co...,Ordonnance N°99-032/P-RM,droit minier,1999,modifies
1,Décret n°2000-050 (10.02.2000) Modification Co...,Ordonnance N°00-013/P-RM,droit minier,2000,amends
2,Décret n°2000-050 (10.02.2000) Modification Co...,Décret N°99-255/P-RM,droit minier,1999,fixes modalities of application
3,Décret n°2000-050 (10.02.2000) Modification Co...,Décret N°97-263/P-RM,droit minier,1997,nominates Prime Minister
4,Décret n°2000-050 (10.02.2000) Modification Co...,Décret N°97-282/P-RM,droit minier,1997,nominates Government members
...,...,...,...,...,...
267,Ordonnance n°1970-006 (27.02.1970) Code généra...,Code général des impôts 1999,fiscal,1999,modifie
268,Ordonnance n°2000-013 (10.02.2000) Modificatio...,Modification Code minier 2000 (SGG),Droit minier,2000,modifie
269,img_Décret n°1999-256 (15.09.1999) Convention ...,Convention minière type 1999,Droit minier,1999,modifie
270,Loi n°1994-034 (09.06.1994) Modification CGI 1...,Modification CGI,Droit fiscal,1994,modifie


In [47]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import re

In [72]:
documents_0 = [re.split(pattern=r' ', string=doc) for doc in df_TexteCodeArborescence[0].unique()]

documents_1 = [re.split(pattern=r' ', string=doc) for doc in df_TexteCodeArborescence[[1,2,3]].agg(' '.join, axis=1)]

documents = [TaggedDocument(words=doc, tags=[" ".join(doc)]) for doc in documents_0] + \
    [TaggedDocument(words=doc, tags=[" ".join(doc)]) for doc in documents_1]

In [371]:
# Initialize the Doc2Vec model
model = Doc2Vec(vector_size=32,  # Dimensionality of the document vectors
                window=2,         # Maximum distance between the current and predicted word within a sentence
                min_count=0,      # Ignores all words with total frequency lower than this
                workers=32,        # Number of CPU cores to use for training
                epochs=15)        # Number of training epochs

# Build the vocabulary
model.build_vocab(documents)

In [372]:
# Train the model
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

In [370]:
inferred_vectors_0 = [model.infer_vector(doc) for doc in documents_0]
inferred_vectors_1 = [model.infer_vector(doc) for doc in documents_1]

In [201]:
import numpy as np

In [86]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(x, y):
    return dot(x, y)/(norm(x)*norm(y))

In [356]:
len(model.infer_vector(documents_0[2]))

1024

In [357]:
model.infer_vector(documents_1[2])

array([ 0.00103045, -0.00381627, -0.00142424, ..., -0.00389062,
        0.00299835, -0.00053239], dtype=float32)

In [387]:
cosine_similarity(model.infer_vector(documents_0[2]), model.infer_vector(documents_1[2]))

0.8752747

In [401]:
idx = 4

similarity = np.abs([cosine_similarity(inferred_vectors_1[idx], y) for y in inferred_vectors_0])

print(max(similarity))
print(df_TexteCodeArborescence[0].unique()[idx])
print(df_TexteCodeArborescence[0].unique()[np.argmax(similarity)])

print(df_TexteCodeArborescence[[1,2,3]].agg(' '.join, axis=1)[idx])

0.6719121725766162
Loi n°2011-078 (23.12.2011) Annexe fiscale Loi de finances 2012 (SEAG Conseil)
img_Décret n°1999-255 (15.09.1999) Application Code minier 1999 (Ministère des Mines)
Décret N°97-282/P-RM droit minier 1997


In [405]:
docs = list()
similarities = list()

for idx in range(len(inferred_vectors_1)):
    similarity = np.abs([cosine_similarity(inferred_vectors_1[idx], y) for y in inferred_vectors_0])

    # print(max(similarity))
    similarities.append(max(similarity))
    docs.append(df_TexteCodeArborescence[0].unique()[np.argmax(similarity)])
    # print(df_TexteCodeArborescence[0].unique()[idx])
    # print(df_TexteCodeArborescence[0].unique()[np.argmax(similarity)])

    # print(df_TexteCodeArborescence[[1,2,3]].agg(' '.join, axis=1)[idx])

In [407]:
df_TexteCodeArborescence['related_doc'] = docs
df_TexteCodeArborescence['similarity'] = similarities

In [409]:
df_TexteCodeArborescence.to_csv(f"../output/TexteCodeArborescence_{COUNTRY_NAME}.csv", index=True)

## Voyage AI Embeddings

In [394]:
from llm_setup import EMBEDDING_MODEL_SETTINGS, VOYAGE_API_KEY
import voyageai

vo = voyageai.Client(api_key=VOYAGE_API_KEY)
# This will automatically use the environment variable VOYAGE_API_KEY.
# Alternatively, you can use vo = voyageai.Client(api_key="<your secret key>")

texts = list(df_TexteCodeArborescence[0].unique())

# Embed the documents
inferred_vectors_0 = vo.embed(list(df_TexteCodeArborescence[0].unique()), model=EMBEDDING_MODEL_SETTINGS['VOYAGE_AI']['name'], input_type="document").embeddings

BATCH_SIZE = 128

related_docs = df_TexteCodeArborescence[[1,2,3]].agg(' '.join, axis=1).to_list()
inferred_vectors_1 = []

for idx in range(0,len(related_docs), BATCH_SIZE):
    inferred_vectors_1.extend(vo.embed(related_docs[slice(idx, (idx + 1) * BATCH_SIZE)], model=EMBEDDING_MODEL_SETTINGS['VOYAGE_AI']['name'], input_type="document").embeddings)

## Open AI similarity

In [410]:
from openai import OpenAI
client = OpenAI()


In [414]:
related_docs = df_TexteCodeArborescence[[1,2,3]].agg(' '.join, axis=1).to_list()

legal_docs = list(df_TexteCodeArborescence[0].unique())

In [428]:
df_TexteCodeArborescence[0].to_list()

['Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (SGG)',
 'Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (SGG)',
 'Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (SGG)',
 'Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (SGG)',
 'Décret n°2000-050 (10.02.2000) Modification Convention minière type 2000 (SGG)',
 'img_Loi n°2004-037 (02.08.2004) Code pétrolier 2004 (Droit-Afrique)',
 'img_Décret n°2018-595 (24.07.2018) Application Cadre général des exonérations fiscales et douanières 2018 (DGI)',
 'img_Décret n°2018-595 (24.07.2018) Application Cadre général des exonérations fiscales et douanières 2018 (DGI)',
 'img_Décret n°2018-595 (24.07.2018) Application Cadre général des exonérations fiscales et douanières 2018 (DGI)',
 'img_Décret n°2018-595 (24.07.2018) Application Cadre général des exonérations fiscales et douanières 2018 (DGI)',
 'img_Décret n°2018-595 (24.07.2018) Application 

In [432]:
related_legal_docs = list()

for (idx, related_doc) in enumerate(related_docs):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant who can read and analyze legal documents."},
            {
                "role": "user",
                "content": f"""
                Consider this title '{doc}'. 
                Find the most similar title from this list {set(legal_docs) - set(df_TexteCodeArborescence[0].to_list()[idx])}.
                Respond only with the title.
                If it does not match any title from the list, respond with None."""
            }
        ]
    )

    print(completion.choices[0].message.content)
    related_legal_docs.append(completion.choices[0].message.content)

img_Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)
Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)
Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)
Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)
Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)
Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)
Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)
Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)
Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)
img_Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)
Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)
Loi n°2014-056 (26.12.2014) Annexe fiscale Loi de finances 2015 (Droit-Afrique)
Loi n°2014-056 (26.12.2014) Anne

KeyboardInterrupt: 

## Load existing categories

In [24]:
COUNTRY_NAME = 'BEN Bénin'

large_categs = pd.read_csv(f"output/{table_column_name}_{COUNTRY_NAME}.csv")
large_categs = list(list(large_categs.set_index(['Unnamed: 0']).to_dict().values())[0].items())

## Update column in table

In [25]:
from postgres_connection import bulk_update_textes

In [26]:
# bulk_update_employees(textes_to_update = large_categs[0:1], )

bulk_update_textes(
    textes_to_update = large_categs, 
    column_name = table_column_name, 
    COUNTRY_NAME=COUNTRY_NAME)

Ordonnance n°1966-002 (10.01.1966) Code général des impôts 2010 (eRegularisation Bénin) déverrouillé column TexteCourt updated to Code général des impôts 2010, Ordonnance n°1966-002 de 1966.
img_Loi n°2005-042 (27.12.2006) Loi de finances 2006 (LégiBénin) column TexteCourt updated to Loi de finances 2006, Modification 2005 du Code général des impôts..
img_Loi n°2007-033 (02.01.2008) Loi de finances 2008 (LégiBénin) column TexteCourt updated to Loi de finances 2008, Loi de finances 2007.
Loi n°2005-016 (08.09.2005) Zone franche industrielle 2005 (Droit-Afrique) column TexteCourt updated to Loi sur la zone franche industrielle 2005, Loi n° 99-001 de 1999.
Liste des lois (Assemblée Nationale) column TexteCourt updated to Liste des lois 2006, Compilation des lois votées et promulguées 1990.
img_Loi n°2006-024 (26.12.2006) Loi de finances 2007 (LégiBénin) column TexteCourt updated to Loi de finances 2007, Modification 2006 de la Loi de finances 2007.
img_Loi n°1997-001 (21.01.1997) Loi de f